In [1]:
import pandas as pd

In [2]:
filmes = pd.read_csv("../../../dados/imdb-reviews-pt-br.csv")

In [11]:
filmes["classify"] = filmes["sentiment"].replace(["neg", "pos"], [0, 1])

In [4]:
# from sklearn.linear_model import LogisticRegression
# reg_log = LogisticRegression()

In [166]:
filmes["text_minus"] = filmes["text_pt"].str.lower()

In [173]:
filmes["text_no_enter"] = filmes["text_minus"].replace(["\n", "\r", "\t"], [' ', ' ', ' '])

In [207]:
import string
mytrans = str.maketrans("", "", string.punctuation)
filmes["text_no_punct"] = filmes["text_no_enter"].str.translate(mytrans)

In [299]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Antonio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [300]:
palavras_adicionais = ['então', 'apenas', 'história', 'enquanto', 'ainda', 'agora', 'porque']
stop_words.extend(palavras_adicionais)
stop_words.sort()

In [302]:
def remove_stop_words( texto ): 
    global stop_words
    lista_palavras = texto.split(" ")
    nova_lista = []
    for palavra in lista_palavras:
        if palavra not in stop_words:
            nova_lista.append(palavra)
    return " ".join(nova_lista)

In [303]:
filmes["texto_no_stopwords"] = filmes["text_no_punct"].apply(remove_stop_words)

In [304]:
# filmes["text_no_punct"].iloc[0]

In [305]:
# remove_stop_words(filmes["text_no_punct"].iloc[0])

In [306]:
textos = filmes["texto_no_stopwords"]

In [307]:
from sklearn.feature_extraction.text import CountVectorizer
vetorizador = CountVectorizer(lowercase=False, max_features=200)
bow = vetorizador.fit_transform(textos)
vocabulario = vetorizador.get_feature_names_out()
bow_matrix = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizador.get_feature_names_out())

In [308]:
# Separar o DataSet em treinamento e testes

In [309]:
# lote_treinamento_size = 25000
# filmes_treino = bow[0:lote_treinamento_size]
# filmes_treino_classe = filmes["classify"].iloc[0:lote_treinamento_size]
# filmes_teste = bow[lote_treinamento_size:49459]
# filmes_teste_classe = filmes["classify"].iloc[lote_treinamento_size:49459]

In [310]:
from sklearn.model_selection import train_test_split
filmes_treino, filmes_teste, filmes_treino_classe, filmes_teste_classe = \
train_test_split(bow_matrix, filmes["classify"], random_state=10)

In [311]:
from sklearn.linear_model import LogisticRegression
reg_log = LogisticRegression(solver="newton-cholesky")
reg_log.fit(filmes_treino, filmes_treino_classe)

LogisticRegression(solver='newton-cholesky')

In [312]:
reg_log.score(filmes_teste, filmes_teste_classe)

0.7767893247068338

In [276]:
vocabulario.shape

(200,)

In [277]:
vetorizador_my_words = CountVectorizer(vocabulary = vocabulario, max_features=200)

In [278]:
meu_bow = vetorizador_my_words.fit_transform(["pessimo filme, perdi tempo assistindo"])

In [279]:
meu_bow.shape

(1, 200)

In [280]:
resultado = reg_log.predict( meu_bow )

C:\Users\Antonio\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [281]:
resultado[0]

1

### Treinamento com o conjunto de dados selecionados pelo train_test_split

### Treinamento com o solver='newton-cholesky'